<a href="https://colab.research.google.com/github/sovunia-hub/electronics-company-data-processing-system/blob/main/DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

 * Starting PostgreSQL 14 database server
   ...done.
ERROR:  role "root" already exists


In [ ]:
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
!kaggle datasets download cameronseamons/electronic-sales-sep2023-sep2024
!unzip electronic-sales-sep2023-sep2024.zip

Dataset URL: https://www.kaggle.com/datasets/cameronseamons/electronic-sales-sep2023-sep2024
License(s): MIT
electronic-sales-sep2023-sep2024.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  electronic-sales-sep2023-sep2024.zip
replace Electronic_sales_Sep2023-Sep2024.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Electronic_sales_Sep2023-Sep2024.csv  


In [ ]:
import pandas as pd

path = 'Electronic_sales_Sep2023-Sep2024.csv'

data = pd.read_csv(path)
data.head()

,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,Unit Price,Quantity,Purchase Date,Shipping Type,Add-ons Purchased,Add-on Total
0,1000,53,Male,No,Smartphone,SKU1004,2,Cancelled,Credit Card,5538.33,791.19,7,2024-03-20,Standard,"Accessory,Accessory,Accessory",40.21
1,1000,53,Male,No,Tablet,SKU1002,3,Completed,Paypal,741.09,247.03,3,2024-04-20,Overnight,Impulse Item,26.09
2,1002,41,Male,No,Laptop,SKU1005,3,Completed,Credit Card,1855.84,463.96,4,2023-10-17,Express,NaN,0.00
3,1002,41,Male,Yes,Smartphone,SKU1004,2,Completed,Cash,3164.76,791.19,4,2024-08-09,Overnight,"Impulse Item,Impulse Item",60.16
4,1003,75,Male,Yes,Smartphone,SKU1001,5,Completed,Cash,41.50,20.75,2,2024-05-21,Express,Accessory,35.56


In [ ]:
data.columns

Index(['Customer ID', 'Age', 'Gender', 'Loyalty Member', 'Product Type', 'SKU',
       'Rating', 'Order Status', 'Payment Method', 'Total Price', 'Unit Price',
       'Quantity', 'Purchase Date', 'Shipping Type', 'Add-ons Purchased',
       'Add-on Total'],
      dtype='object')

In [ ]:
data.columns = ['Customer_id', 'Age', 'Gender', 'Is_loyalty_member', 'Type', 'Product_id',
       'Rating', 'Status', 'Payment', 'Total_price', 'Price',
       'Quantity', 'Date', 'Shipping_type', 'Add-ons_purchased',
       'Add-ons_total']

In [ ]:
data.dtypes

,0
Customer_id,int64
Age,int64
Gender,object
Is_loyalty_member,object
Type,object
Product_id,object
Rating,int64
Status,object
Payment,object
Total_price,float64


In [ ]:
customers = data[['Customer_id', 'Age', 'Gender', 'Is_loyalty_member']]
customers = customers.drop_duplicates(['Customer_id'], keep = 'last').reset_index(drop=True)
customers['Is_loyalty_member'] = customers['Is_loyalty_member'].apply(lambda x: True if x == 'Yes' else False)
customers.head()

,Customer_id,Age,Gender,Is_loyalty_member
0,1000,53,Male,False
1,1002,41,Male,True
2,1003,75,Male,True
3,1004,41,Female,False
4,1005,25,Female,False


In [ ]:
orders = data[['Customer_id', 'Status', 'Payment', 'Total_price', 'Date', 'Shipping_type']]
orders = orders.rename_axis('Order_id').reset_index()
orders.Date = pd.to_datetime(orders.Date)
orders.head()

,Order_id,Customer_id,Status,Payment,Total_price,Date,Shipping_type
0,0,1000,Cancelled,Credit Card,5538.33,2024-03-20,Standard
1,1,1000,Completed,Paypal,741.09,2024-04-20,Overnight
2,2,1002,Completed,Credit Card,1855.84,2023-10-17,Express
3,3,1002,Completed,Cash,3164.76,2024-08-09,Overnight
4,4,1003,Completed,Cash,41.50,2024-05-21,Express


In [ ]:
order_items = data[['Product_id', 'Quantity', 'Add-ons_purchased', 'Add-ons_total']]
order_items = order_items.rename_axis('Order_id').reset_index()
order_items.head()

,Order_id,Product_id,Quantity,Add-ons_purchased,Add-ons_total
0,0,SKU1004,7,"Accessory,Accessory,Accessory",40.21
1,1,SKU1002,3,Impulse Item,26.09
2,2,SKU1005,4,NaN,0.00
3,3,SKU1004,4,"Impulse Item,Impulse Item",60.16
4,4,SKU1001,2,Accessory,35.56


In [ ]:
products = data[['Product_id', 'Type', 'Price', 'Rating']]
products = products.drop_duplicates().reset_index(drop=True)
products.head()

,Product_id,Type,Price,Rating
0,SKU1004,Smartphone,791.19,2
1,SKU1002,Tablet,247.03,3
2,SKU1005,Laptop,463.96,3
3,SKU1001,Smartphone,20.75,5
4,SKU1003,Smartwatch,844.83,3


In [ ]:
from sqlalchemy import create_engine

engine = create_engine('postgresql+psycopg2://@/postgres')

customers.to_sql('customers', engine, if_exists='replace', index=False)
orders.to_sql('orders', engine, if_exists='replace', index=False)
order_items.to_sql('order_items', engine, if_exists='replace', index=False)
products.to_sql('products', engine, if_exists='replace', index=False)

34

In [ ]:
import prettytable

prettytable.DEFAULT = 'DEFAULT'

In [ ]:
%sql SELECT * FROM orders LIMIT 5;

 * postgresql+psycopg2://@/postgres


,Order_id,Customer_id,Status,Payment,Total_price,Date,Shipping_type
0,0,1000,Cancelled,Credit Card,5538.33,2024-03-20,Standard
1,1,1000,Completed,Paypal,741.09,2024-04-20,Overnight
2,2,1002,Completed,Credit Card,1855.84,2023-10-17,Express
3,3,1002,Completed,Cash,3164.76,2024-08-09,Overnight
4,4,1003,Completed,Cash,41.50,2024-05-21,Express


# Простые триггеры

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION check_age()
RETURNS TRIGGER AS $$
BEGIN
  IF NEW."Age" < 14 THEN
    RAISE EXCEPTION 'Возраст клиента не может быть менее 14 лет';
  END IF;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER age_check_trigger
BEFORE INSERT OR UPDATE ON customers
FOR EACH ROW EXECUTE FUNCTION check_age();

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql INSERT INTO customers VALUES (1, 10, 'Male', True);

 * postgresql+psycopg2://@/postgres
(psycopg2.errors.RaiseException) Возраст клиента не может быть менее 14 лет
CONTEXT:  PL/pgSQL function check_age() line 4 at RAISE

[SQL: INSERT INTO customers VALUES (1, 10, 'Male' , True);]
(Background on this error at: https://sqlalche.me/e/20/2j85)


In [ ]:
%%sql
CREATE OR REPLACE FUNCTION set_current_date()
RETURNS TRIGGER AS $$
BEGIN
  IF (NEW."Date" IS NULL) OR (NEW."Date" > NOW()) THEN
    NEW."Date" = NOW();
  END IF;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER set_date_trigger
BEFORE INSERT ON orders
FOR EACH ROW EXECUTE FUNCTION set_current_date();

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql INSERT INTO orders VALUES (1, 1, 'Cancelled', 'Paypal', 433.98, NULL, 'Overnight');

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM orders WHERE "Customer_id" = 1;

 * postgresql+psycopg2://@/postgres


,Order_id,Customer_id,Status,Payment,Total_price,Date,Shipping_type
0,1,1,Cancelled,Paypal,433.98,2024-12-11 22:14:50.094611,Overnight


In [ ]:
%%sql
CREATE OR REPLACE FUNCTION check_rating()
RETURNS TRIGGER AS $$
BEGIN
  IF NEW."Rating" < 1 OR NEW."Rating" > 5 THEN
    RAISE EXCEPTION 'Рейтинг должен быть в диапазоне от 1 до 5';
  END IF;
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER rating_check_trigger
BEFORE INSERT OR UPDATE ON products
FOR EACH ROW EXECUTE FUNCTION check_rating();

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql INSERT INTO products VALUES ('UDP821', 'Smartphone',	484.22, 3934);

 * postgresql+psycopg2://@/postgres
(psycopg2.errors.RaiseException) Рейтинг должен быть в диапазоне от 1 до 5
CONTEXT:  PL/pgSQL function check_rating() line 4 at RAISE

[SQL: INSERT INTO products VALUES ('UDP821', 'Smartphone' , 484.22, 3934);]
(Background on this error at: https://sqlalche.me/e/20/2j85)


# Сложные триггеры

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION calculate_order_total()
RETURNS TRIGGER AS $$
DECLARE
  total DECIMAL(10, 2) := 0;
BEGIN
  IF NOT EXISTS (SELECT 1 FROM orders WHERE "Order_id" = NEW."Order_id") THEN
    INSERT INTO orders ("Order_id")
    VALUES (NEW."Order_id");
  END IF;
  SELECT SUM(p."Price") * NEW."Quantity" + COALESCE(NEW."Add-ons_total", 0)
  INTO total
  FROM products p
  WHERE p."Product_id" = NEW."Product_id";

  UPDATE orders
  SET "Total_price" = total
  WHERE "Order_id" = NEW."Order_id";

  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER calculate_total_trigger
BEFORE INSERT OR UPDATE ON order_items
FOR EACH ROW EXECUTE FUNCTION calculate_order_total();

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM orders WHERE "Order_id" = 0;

 * postgresql+psycopg2://@/postgres


,Order_id,Customer_id,Status,Payment,Total_price,Date,Shipping_type
0,0,1000,Cancelled,Credit Card,5538.33,2024-03-20,Standard


In [ ]:
%sql INSERT INTO order_items VALUES (0, 'SKU1004', 7, 'Accessory,Accessory,Accessory', 40.21);

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM orders WHERE "Order_id" = 0;

 * postgresql+psycopg2://@/postgres


,Order_id,Customer_id,Status,Payment,Total_price,Date,Shipping_type
0,0,1000,Cancelled,Credit Card,17470.35,2024-03-20,Standard


In [ ]:
%%sql
CREATE OR REPLACE FUNCTION handle_order_deletion()
RETURNS TRIGGER AS $$
BEGIN
  DELETE FROM order_items
  WHERE "Order_id" = OLD."Order_id";

  RETURN OLD;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER order_deletion_trigger
AFTER DELETE ON orders
FOR EACH ROW EXECUTE FUNCTION handle_order_deletion();

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql DELETE FROM orders WHERE "Order_id" = 0;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM order_items WHERE "Order_id" = 0;

 * postgresql+psycopg2://@/postgres


""


# Простые процедуры

## add_order_item

In [ ]:
%%sql
CREATE OR REPLACE PROCEDURE add_order_item(
  p_order_id INT,
  p_product_id VARCHAR
)
LANGUAGE plpgsql
AS $$
DECLARE
  affected_rows INT;
BEGIN
  UPDATE order_items
  SET "Quantity" = "Quantity" + 1
  WHERE "Product_id" = p_product_id AND "Order_id" = p_order_id;

  GET DIAGNOSTICS affected_rows = ROW_COUNT;

  IF affected_rows = 0 THEN
    INSERT INTO order_items
    VALUES (p_order_id, p_product_id, 1, '', 0);
  END IF;
END;
$$;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM order_items WHERE "Order_id" = 1;

 * postgresql+psycopg2://@/postgres


,Order_id,Product_id,Quantity,Add-ons_purchased,Add-ons_total
0,1,SKU1002,3,Impulse Item,26.09


In [ ]:
%sql CALL add_order_item(1, 'SKU1004')

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM order_items WHERE "Order_id" = 1;

 * postgresql+psycopg2://@/postgres


,Order_id,Product_id,Quantity,Add-ons_purchased,Add-ons_total
0,1,SKU1002,3,Impulse Item,26.09
1,1,SKU1004,1,,0.00


## complete_order

In [ ]:
%%sql
CREATE OR REPLACE PROCEDURE complete_order(
  p_order_id INT,
  p_customer_id INT,
  p_payment VARCHAR,
  p_shipping_type VARCHAR
)
LANGUAGE plpgsql
AS $$
BEGIN
  UPDATE orders
  SET "Payment" = p_payment, "Shipping_type" = p_shipping_type, "Status" = 'Completed'
  WHERE "Customer_id" = p_customer_id AND "Order_id" = p_order_id;
END;
$$;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM orders WHERE "Order_id" = 1;

 * postgresql+psycopg2://@/postgres


,Order_id,Customer_id,Status,Payment,Total_price,Date,Shipping_type
0,1,1000,Completed,Paypal,2490.02,2024-04-20 00:00:00.000000,Overnight
1,1,1,Cancelled,Paypal,2490.02,2024-12-11 22:14:50.094611,Overnight


In [ ]:
%sql CALL complete_order(1, 1, 'Paypal', 'Overnight')

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM orders WHERE "Order_id" = 1;

 * postgresql+psycopg2://@/postgres


,Order_id,Customer_id,Status,Payment,Total_price,Date,Shipping_type
0,1,1000,Completed,Paypal,2490.02,2024-04-20 00:00:00.000000,Overnight
1,1,1,Completed,Paypal,2490.02,2024-12-11 22:14:50.094611,Overnight


## add_customer

In [ ]:
%%sql
CREATE OR REPLACE PROCEDURE add_customer(
  p_customer_id INT,
  p_age INT,
  p_gender VARCHAR,
  p_is_loyalty_member BOOLEAN
)
LANGUAGE plpgsql
AS $$
BEGIN
  INSERT INTO customers VALUES (p_customer_id, p_age, p_gender, p_is_loyalty_member);
END;
$$;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql CALL add_customer(999, 29, 'Male', False)

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM customers WHERE "Customer_id" = 999;

 * postgresql+psycopg2://@/postgres


,Customer_id,Age,Gender,Is_loyalty_member
0,999,29,Male,False


## add_product

In [ ]:
%%sql
CREATE OR REPLACE PROCEDURE add_product(
  p_product_id VARCHAR,
  p_type VARCHAR,
  p_price DECIMAL(10,2),
  p_rating INT
)
LANGUAGE plpgsql
AS $$
BEGIN
  INSERT INTO products VALUES (p_product_id, p_type, p_price, p_rating);
END;
$$;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql CALL add_product('SUK783', 'Smarphone', 142.99, 4)

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM products WHERE "Product_id" = 'SUK783';

 * postgresql+psycopg2://@/postgres


,Product_id,Type,Price,Rating
0,SUK783,Smarphone,142.99,4


## update_product_info

In [ ]:
%%sql
CREATE OR REPLACE PROCEDURE update_product_info(
  product_id VARCHAR,
  new_price DECIMAL(10,2),
  new_rating INT
)
LANGUAGE plpgsql AS $$
BEGIN
  UPDATE products
  SET
    "Price" = new_price,
    "Rating" = new_rating
  WHERE
    "Product_id" = product_id;
  IF NOT FOUND THEN
    RAISE NOTICE 'Продукт с ID % не найден.', product_id;
  END IF;
END;
$$;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql CALL update_product_info('SUK783', 199.99, 5);

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM products WHERE "Product_id" = 'SUK783';

 * postgresql+psycopg2://@/postgres


,Product_id,Type,Price,Rating
0,SUK783,Smarphone,199.99,5


# Простые функции

In [ ]:
%%sql
DROP FUNCTION get_cart(integer)

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
CREATE OR REPLACE FUNCTION get_cart(
  p_order_id INT
) RETURNS TABLE (
  "Product_id" TEXT,
  "Quantity" bigint,
  "Total_price" numeric
) AS $$
BEGIN
  RETURN QUERY
  SELECT o."Product_id", MIN(o."Quantity"), (MIN(o."Quantity") * MIN(p."Price"))::numeric AS "Total_price"
  FROM products p JOIN order_items o
  ON o."Product_id" = p."Product_id"
  WHERE o."Order_id" = p_order_id
  GROUP BY o."Product_id";
END;
$$ LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM get_cart(1);

 * postgresql+psycopg2://@/postgres


,Product_id,Quantity,Total_price
0,SKU1002,3,741.09
1,SKU1004,1,766.8


## get_customer_orders

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION get_customer_orders(
  p_customer_id INT
) RETURNS TABLE (
  "Order_id" INT,
  "Status" VARCHAR,
  "Total_price" DECIMAL(10,2),
  "Date" DATE
) AS $$
BEGIN
  RETURN QUERY
  SELECT o."Order_id"::INT, o."Status"::VARCHAR, o."Total_price"::DECIMAL(10,2), o."Date"::DATE
  FROM orders o
  WHERE o."Customer_id" = p_customer_id;
END;
$$ LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM get_customer_orders(1002);

 * postgresql+psycopg2://@/postgres


,Order_id,Status,Total_price,Date
0,2,Completed,1855.84,2023-10-17
1,3,Completed,3164.76,2024-08-09


## customer_exists

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION customer_exists(
  p_customer_id INT
) RETURNS BOOLEAN AS $$
DECLARE
  customer_count INT;
BEGIN
  SELECT COUNT(*)
  INTO customer_count
  FROM customers
  WHERE "Customer_id" = p_customer_id;

  RETURN customer_count > 0;
END;
$$ LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM customer_exists(1002);

 * postgresql+psycopg2://@/postgres


,customer_exists
0,True


## get_order_statistics

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION get_order_statistics(
  p_customer_id INT
) RETURNS TABLE (
  Total_orders INT,
  Total_spent NUMERIC,
  Average_order_value NUMERIC
) AS $$
BEGIN
  RETURN QUERY
  SELECT
    COUNT(*)::INT AS Total_orders,
    SUM("Total_price")::NUMERIC AS Total_spent,
    AVG("Total_price")::NUMERIC AS Average_order_value
  FROM orders
  WHERE "Customer_id" = p_customer_id;
END;
$$ LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM get_order_statistics(1006)

 * postgresql+psycopg2://@/postgres


,total_orders,total_spent,average_order_value
0,3,12184.27,4061.42333333333


## get_top_products

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION get_top_products(
  p_limit INT
) RETURNS TABLE (
  Product_id VARCHAR,
  Product_type VARCHAR,
  Total_quantity INT
) AS $$
BEGIN
  RETURN QUERY
  SELECT
    oi."Product_id"::VARCHAR,
    p."Type"::VARCHAR AS Product_type,
    SUM(oi."Quantity")::INT AS Total_quantity
  FROM order_items oi
  JOIN products p ON oi."Product_id" = p."Product_id"
  GROUP BY oi."Product_id", p."Type"
  ORDER BY Total_quantity DESC
  LIMIT p_limit;
END;
$$ LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM get_top_products(5)

 * postgresql+psycopg2://@/postgres


,product_id,product_type,total_quantity
0,TBL345,Tablet,56695
1,HDP456,Headphones,55890
2,SWT567,Smartwatch,54995
3,SMP234,Smartphone,54630
4,LTP123,Laptop,54210


## get_max_order_value

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION get_max_order_value(
  p_customer_id INT
) RETURNS INT AS $$
DECLARE
  max_order_value INT;
BEGIN
  SELECT MAX("Total_price") INTO max_order_value
  FROM orders
  WHERE "Customer_id" = p_customer_id;

  RETURN COALESCE(max_order_value, 0);
END;
$$ LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT get_max_order_value(1006)

 * postgresql+psycopg2://@/postgres


,get_max_order_value
0,5538


## get_unique_products_count

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION get_unique_products_count(
  p_customer_id INT
) RETURNS INT AS $$
DECLARE
  unique_products_count INT;
BEGIN
  SELECT COUNT(DISTINCT oi."Product_id") INTO unique_products_count
  FROM orders o
  JOIN order_items oi ON o."Order_id" = oi."Order_id"
  WHERE o."Customer_id" = p_customer_id;

  RETURN unique_products_count;
END;
$$ LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT get_unique_products_count(1006)

 * postgresql+psycopg2://@/postgres


,get_unique_products_count
0,3


## add_order

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION add_order(p_customer_id INT)
RETURNS INT AS $$
DECLARE
  new_order_id INT;
BEGIN
  SELECT MAX("Order_id") FROM Orders INTO new_order_id;
  new_order_id:= new_order_id + 1;

  INSERT INTO orders ("Order_id", "Customer_id", "Date", "Status")
  VALUES (new_order_id, p_customer_id, NOW(), 'In progress');

  RETURN new_order_id;
END;
$$ LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT add_order(1000)

 * postgresql+psycopg2://@/postgres


,add_order
0,20001


## add_customer

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION add_customer(
  p_age INT,
  p_gender VARCHAR
)
RETURNS INT AS $$
DECLARE
  new_customer_id INT;
BEGIN
  SELECT MAX("Customer_id") FROM customers INTO new_customer_id;
  new_customer_id:= new_customer_id + 1;
  INSERT INTO customers ("Customer_id", "Age", "Gender")
  VALUES (new_customer_id, p_age, p_gender);

  RETURN new_customer_id;
END;
$$ LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT add_customer(53, 'Male')

 * postgresql+psycopg2://@/postgres


,add_customer
0,20000


In [ ]:
%sql SELECT * FROM customers

 * postgresql+psycopg2://@/postgres


,Customer_id,Age,Gender,Is_loyalty_member
0,1000.0,53,Male,False
1,1002.0,41,Male,True
2,1003.0,75,Male,True
3,1004.0,41,Female,False
4,1005.0,25,Female,False
...,...,...,...,...
24269,NaN,56,Male,None
24270,NaN,56,Male,None
24271,NaN,56,Male,None
24272,NaN,56,Male,None


# Представления

In [ ]:
%%sql
CREATE OR REPLACE VIEW vw_full_order_details AS
SELECT
  o."Order_id",
  o."Customer_id",
  c."Gender",
  c."Age",
  o."Status",
  o."Total_price",
  o."Date" AS Order_date,
  oi."Product_id",
  p."Type" AS Product_type,
  oi."Quantity",
  oi."Add-ons_purchased",
  oi."Add-ons_total"
FROM orders o
JOIN customers c ON o."Customer_id" = c."Customer_id"
JOIN order_items oi ON o."Order_id" = oi."Order_id"
JOIN products p ON oi."Product_id" = p."Product_id";

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM vw_full_order_details LIMIT 5;

 * postgresql+psycopg2://@/postgres


,Order_id,Customer_id,Gender,Age,Status,Total_price,order_date,Product_id,product_type,Quantity,Add-ons_purchased,Add-ons_total
0,1,1000,Male,53,Completed,741.09,2024-04-20,SKU1002,Tablet,3,Impulse Item,26.09
1,2,1002,Male,41,Completed,1855.84,2023-10-17,SKU1005,Laptop,4,None,0.00
2,2,1002,Male,41,Completed,1855.84,2023-10-17,SKU1005,Smartphone,4,None,0.00
3,3,1002,Male,41,Completed,3164.76,2024-08-09,SKU1004,Smartphone,4,"Impulse Item,Impulse Item",60.16
4,3,1002,Male,41,Completed,3164.76,2024-08-09,SKU1004,Laptop,4,"Impulse Item,Impulse Item",60.16


In [ ]:
%%sql
CREATE OR REPLACE VIEW vw_cancelled_orders AS
SELECT
  o."Order_id",
  o."Customer_id",
  o."Status",
  o."Total_price",
  o."Date" AS Order_date
FROM orders o
JOIN customers c ON o."Customer_id" = c."Customer_id"
WHERE o."Status" = 'Cancelled';

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM vw_cancelled_orders LIMIT 5;

 * postgresql+psycopg2://@/postgres


,Order_id,Customer_id,Status,Total_price,order_date
0,8,1006,Cancelled,5538.33,2023-10-03
1,11,1007,Cancelled,7120.71,2023-11-07
2,14,1013,Cancelled,5538.33,2024-03-18
3,15,1014,Cancelled,5068.98,2024-01-02
4,20,1019,Cancelled,83.00,2024-03-14


In [ ]:
%%sql
CREATE OR REPLACE VIEW vw_customer_statistics AS
SELECT
  c."Customer_id",
  COUNT(o."Order_id") AS Total_orders,
  COALESCE(SUM(o."Total_price"), 0) AS Total_spent
FROM customers c
LEFT JOIN orders o ON c."Customer_id" = o."Customer_id"
GROUP BY c."Customer_id"
ORDER BY "Customer_id";

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM vw_customer_statistics LIMIT 5;

 * postgresql+psycopg2://@/postgres


,Customer_id,total_orders,total_spent
0,999,0,0.00
1,1000,1,741.09
2,1002,2,5020.60
3,1003,1,41.50
4,1004,1,83.00
